In [1]:
from geoalchemy2 import Geometry, load_spatialite
from flask_caching import Cache
from sqlalchemy import event, Table, insert, func
from sqlalchemy.orm import sessionmaker
import os
from flask import Flask
from quickannotator.db import Project, Image, AnnotationClass, Notification, Tile, Setting, Annotation
from quickannotator.db import db
import large_image
import math
import numpy as np

In [2]:
def insert_project(app, db, name, description, is_dataset_large):
    with app.app_context():
        project = Project(name=name,
                          description=description,
                          is_dataset_large=is_dataset_large)
        db.session.add(project)
        db.session.commit()
        
def insert_image(app, db, project_id, name, path, height, width, dz_tilesize, embedding_coord, group_id, split):
    with app.app_context():
        image = Image(project_id=project_id,
                      name=name,
                      path=path,
                      height=height,
                      width=width,
                      dz_tilesize=dz_tilesize,
                      embedding_coord=embedding_coord,
                      group_id=group_id,
                      split=split)
        db.session.add(image)
        db.session.commit()
        
def insert_annotation_class(app, db, project_id, name, color, magnification, patchsize, tilesize, dl_model_objectref):
    with app.app_context():
        annotation_class = AnnotationClass(project_id=project_id,
                                           name=name,
                                           color=color,
                                           magnification=magnification,
                                           patchsize=patchsize,
                                           tilesize=tilesize,
                                           dl_model_objectref=dl_model_objectref)
        db.session.add(annotation_class)
        db.session.commit()
              
def insert_tile(app, db, image_id, annotation_class_id, geom, seen):
    with app.app_context():
        tile = Tile(image_id=image_id,
                    annotation_class_id=annotation_class_id,
                    geom=geom,
                    seen=seen)
        
        db.session.add(tile)
        db.session.commit()
        
def insert_tiles(app, db, image_id, annotation_class_id):
    image_width, image_height = get_image_dimensions(app, image_id)
    
    with app.app_context():
        tile_size = AnnotationClass.query.filter_by(id=annotation_class_id).first().tilesize
    
    n_cols = math.ceil(image_width / tile_size)
    n_rows = math.ceil(image_height / tile_size)
    
    tiles = []
    for i in range(n_rows):
        for j in range(n_cols):
            t = Tile(image_id=image_id,
                     annotation_class_id=annotation_class_id,
                     geom=f"POLYGON(({j*tile_size} {i*tile_size}, {j*tile_size} {(i+1)*tile_size}, {(j+1)*tile_size} {(i+1)*tile_size}, {(j+1)*tile_size} {i*tile_size}, {j*tile_size} {i*tile_size}))",
                     seen=2
                     )
            tiles.append(t)
    with app.app_context():
        db.session.add_all(tiles)
        db.session.commit()
            
def get_tiles_within_bbox(app, image_id, annotation_class_id, x1y1, x2y2):
    with app.app_context():
        envelope = func.BuildMbr(x1y1[0], x1y1[1], x2y2[0], x2y2[1])
        filter = Tile.geom.ST_intersects(envelope)
        tiles = Tile.query.filter_by(image_id=image_id, annotation_class_id=annotation_class_id).filter(filter).all()
        
    return tiles

    
def get_image_dimensions(app, image_id):
    with app.app_context():
        image = Image.query.filter_by(id=image_id).first()
        return image.width, image.height
        
def create_annotation_table(db, image_id, annotation_class_id, gtpred):
    table_name = f"{image_id}_{annotation_class_id}_{gtpred}_annotation"
    table = Annotation.__table__.to_metadata(db.metadata, name=table_name)
    db.metadata.create_all(bind=db.engine, tables=[table])
    
    
def insert_annotation(app, db, image_id, annotation_class_id, gtpred):
    with app.app_context():
        table_name = f"{image_id}_{annotation_class_id}_{gtpred}_annotation"
        annotation = {"centroid" : "POINT(1 1)",
                                "area" : 1000,
                                "polygon" : "MULTIPOLYGON(((0 0, 0 1, 1 1, 1 0, 0 0)))",
                                "custom_metrics" : {"iou": 0.5}
        }
        
        table = Table(table_name, db.metadata, autoload_with=db.engine)
        stmt = insert(table).values(annotation)
        db.session.execute(stmt)
        db.session.commit()

In [3]:
app = Flask("app")
SearchCache = Cache(config={'CACHE_TYPE': 'SimpleCache'})
SearchCache.init_app(app)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///quickannotator.db'
os.environ['SPATIALITE_LIBRARY_PATH'] = '/usr/lib/x86_64-linux-gnu/mod_spatialite.so'

db.app = app
db.init_app(app)
models = [Project, Image, AnnotationClass, Notification, Tile, Setting]
with app.app_context():
    event.listen(db.engine, 'connect', load_spatialite)
    db.drop_all()
    db.metadata.create_all(bind=db.engine, tables=[item.__table__ for item in models])

    create_annotation_table(db, 1, 1, "gt")

In [4]:
insert_project(app, db, 
               name="example_project", 
               description="test", 
               is_dataset_large=False)

full_path = "/home/jackson/code/research/histotools/QuickAnnotator/quickannotator/data/images/1/TCGA-23-2072-01Z-00-DX1.478243FF-BFF0-48A4-ADEA-DE789331A50E.svs"

path = full_path.split("quickannotator/")[1]

slide = large_image.getTileSource(full_path)
insert_image(app, db, 
             project_id=1, 
             name=os.path.basename(path), 
             path=path, 
             height=slide.sizeY, 
             width=slide.sizeX, 
             dz_tilesize=slide.tileWidth,
             embedding_coord="POINT(1 1)", 
             group_id=1, 
             split=1)

insert_annotation_class(app, db,
                        project_id=None,
                        name="Tissue Mask",
                        color="black",
                        magnification=None,
                        patchsize=None,
                        tilesize=None,
                        dl_model_objectref=None)

insert_annotation_class(app, db,
                        project_id=1,
                        name="Tubule",
                        color="red",
                        magnification=10,
                        patchsize=256,
                        tilesize=2048,
                        dl_model_objectref=None)

insert_annotation(app, db, 1, 1, "gt")

insert_tiles(app, db, image_id=1, annotation_class_id=2)

In [5]:
len(get_tiles_within_bbox(app, 1, 2, [0, 0], [41000, 40000]))

420

In [6]:
get_image_dimensions(app, 1)

(127487, 91863)